# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Deterministic LangGraph Agent with Plan-Execute

**Description:** Ship a production-grade LangGraph agent: deterministic plan-execute, strict JSON schemas, thread memory, SQLite checkpoints, and a single FastAPI /agent endpoint.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction

Building reliable GenAI agents means controlling their behavior. Free-form loops can spiral into unpredictable costs, latencies, and debugging nightmares. This guide shows you how to build a **deterministic plan-execute agent** with LangGraph that fixes the plan upfront, validates every step with strict schemas, and checkpoints progress so you can resume mid-execution. You'll walk away with a working FastAPI service that's debuggable, cost-predictable, and production-ready.

**Prerequisites:** Python 3.10+, an OpenAI API key with billing enabled, and basic familiarity with FastAPI. This tutorial is designed to run end-to-end in a local environment or Colab notebook.

**What you'll build:** A fixed plan → execute → finalize graph with Pydantic schemas, SQLite checkpointing, and a `/agent` endpoint that handles multi-turn conversations with thread-scoped memory.

---

## Why This Approach Works

**Determinism over improvisation.** Traditional ReAct loops let the LLM decide tools and parameters on every turn, leading to drift, redundant calls, and unpredictable token usage. A plan-execute pattern locks in the strategy upfront: the planner emits a fixed sequence of steps, the executor runs them one by one with validated inputs, and the finalizer synthesizes the answer. You get predictable costs, clear traces, and no surprise loops.

**Strict schemas enforce correctness.** Pydantic models validate every tool input and output at runtime. If the planner hallucinates a parameter or the tool returns malformed data, the system fails fast with a typed error—no silent corruption, no downstream crashes.

**Checkpointing enables resumability.** LangGraph's SQLite checkpointer persists state after every node. If a tool times out or the server restarts mid-execution, you resume from the last completed step using the same `thread_id`. This makes the agent resilient to transient failures and safe for long-running workflows.

**Thread-scoped memory keeps context clean.** Each conversation thread maintains its own state and memory summary. You can run multiple concurrent requests without cross-contamination, and the agent can reference prior turns when planning follow-up steps.

---

## How It Works (High-Level Overview)

1. **User sends a request** with a `thread_id` and `user_input` to the `/agent` endpoint.
2. **Planner node** calls OpenAI with function calling to emit a structured `Plan` (title, rationale, list of steps). Each step specifies a tool name, parameters, and description. The plan is capped at 5 steps and uses only allowlisted tools.
3. **Executor node** runs one step per graph visit. It validates the step's parameters against the tool's input schema, calls the tool with retries and timeout enforcement, validates the output, and appends the result to `steps_results`. The graph loops back to executor until all steps are complete.
4. **Finalizer node** sends the accumulated evidence to OpenAI with `response_format=json_object` to generate a concise answer. It packages the answer, evidence, plan summary, and metadata into a `FinalResult`.
5. **Checkpointer** persists state after every node transition. If execution is interrupted, the next request with the same `thread_id` resumes from the last checkpoint.
6. **Memory update** runs after the graph completes, summarizing the final result and storing it in thread-scoped memory for future turns.

**Graph flow:**  
`user_input → plan → execute (loop) → finalize → END`

---

## Setup & Installation

Install dependencies in a single cell (Colab-ready):

In [ ]:
!pip install langgraph langchain-core openai pydantic fastapi uvicorn python-dotenv nest-asyncio

Set your OpenAI API key securely:

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not set. Add it to .env or set it in the environment.")

OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")  # Ensure this model supports tool calling
TEMPERATURE = 0

---

## Step-by-Step Implementation

### Step 1: Define Schemas and State

Use Pydantic to define tool inputs, outputs, plan steps, and agent state. These schemas enforce type safety and enable automatic validation.

In [ ]:
import logging
import sys
from typing import List, Dict, Any, TypedDict
from pydantic import BaseModel, Field

# Set up structured logging
logger = logging.getLogger("agent")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Plan step schema: each step specifies a tool, parameters, and rationale
class PlanStep(BaseModel):
    tool: str = Field(..., description="Tool name from the allowlist")
    params: Dict[str, Any] = Field(default_factory=dict, description="JSON inputs for the tool")
    description: str = Field(..., description="Why this step is needed")

# Plan schema: title, rationale, and a list of steps (max 5)
class Plan(BaseModel):
    title: str
    rationale: str
    steps: List[PlanStep] = Field(..., min_items=1, max_items=5)

# Tool input/output schemas
class SearchInput(BaseModel):
    query: str

class SearchOutput(BaseModel):
    results: List[str]

class RetrieveInput(BaseModel):
    doc_id: str

class RetrieveOutput(BaseModel):
    content: str

class CalcInput(BaseModel):
    expression: str

class CalcOutput(BaseModel):
    value: float

# Final result schema: answer, evidence, plan summary, and metadata
class FinalResult(BaseModel):
    answer: str
    evidence: Dict[str, Any]
    plan_summary: Dict[str, Any]
    metadata: Dict[str, Any]

# Agent state: stores thread_id, user input, plan, execution progress, results, and memory
class AgentState(TypedDict, total=False):
    thread_id: str
    user_input: str
    plan_dict: Dict[str, Any]  # Store plan as dict for checkpoint portability
    step_index: int
    steps_results: List[Dict[str, Any]]
    final_result_dict: Dict[str, Any]  # Store final result as dict
    memory: Dict[str, Any]
    error: Dict[str, Any]  # Structured error payload

**Why dicts in state?** SQLite checkpointer serializes state to JSON. Storing Pydantic models directly can break portability. We store `plan_dict` and `final_result_dict` as plain dicts and convert to/from Pydantic at node boundaries.

### Step 2: Implement Tools with Validation

Define three simple tools: search, retrieve, and calculator. Each tool takes a validated input model and returns a validated output model.

In [ ]:
import ast
import operator
from typing import Callable, Tuple

# In-memory corpus for demonstration
CORPUS = {
    "doc:langgraph": "LangGraph is a library for building stateful, structured LLM workflows.",
    "doc:pydantic": "Pydantic validates data using Python type hints.",
    "doc:fastapi": "FastAPI is a modern, fast web framework for building APIs with Python.",
}

# Safe arithmetic operations for calculator
_ALLOWED_OPS = {
    ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul,
    ast.Div: operator.truediv, ast.Pow: operator.pow, ast.USub: operator.neg
}

def _eval_expr(node):
    """Evaluate a safe arithmetic expression (no variables, no function calls)."""
    if isinstance(node, ast.Num):
        return node.n
    if isinstance(node, ast.UnaryOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_eval_expr(node.operand))
    if isinstance(node, ast.BinOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_eval_expr(node.left), _eval_expr(node.right))
    if isinstance(node, ast.Expression):
        return _eval_expr(node.body)
    raise ValueError("Disallowed expression")

def search_tool(inp: SearchInput) -> SearchOutput:
    """Search tool: finds documents matching the query."""
    q = inp.query.lower().strip()
    hits = [k for k, v in CORPUS.items() if q in v.lower() or q in k.lower()]
    return SearchOutput(results=hits[:5])

def calc_tool(inp: CalcInput) -> CalcOutput:
    """Calculator tool: evaluates arithmetic expressions safely."""
    expr = inp.expression
    node = ast.parse(expr, mode="eval")
    val = float(_eval_expr(node))
    return CalcOutput(value=val)

def retrieve_tool(inp: RetrieveInput) -> RetrieveOutput:
    """Retrieve tool: fetches document content by ID."""
    content = CORPUS.get(inp.doc_id)
    if content is None:
        raise ValueError(f"Document not found: {inp.doc_id}")
    return RetrieveOutput(content=content)

# Tool allowlist: maps tool names to (InputModel, OutputModel, callable)
ToolCallable = Callable[[BaseModel], BaseModel]
ALLOWLIST: Dict[str, Tuple[type[BaseModel], type[BaseModel], ToolCallable]] = {
    "search": (SearchInput, SearchOutput, search_tool),
    "retrieve_doc": (RetrieveInput, RetrieveOutput, retrieve_tool),
    "calculator": (CalcInput, CalcOutput, calc_tool),
}

**Why an allowlist?** It prevents the planner from hallucinating tool names or invoking unsafe operations. Only pre-approved tools can execute.

### Step 3: Build the Planner Node

The planner calls OpenAI with function calling to emit a structured plan. We use `tool_choice` to force the model to call `emit_plan` and return a JSON schema matching `Plan`.

In [ ]:
from openai import OpenAI
from pydantic import ValidationError

client = OpenAI(api_key=OPENAI_API_KEY)

# Planner system prompt: instructs the model to produce a deterministic, finite plan
PLANNER_SYSTEM = """You are a deterministic planning assistant.
- Produce a concise plan to answer the user.
- Use only these tools: search, retrieve_doc, calculator.
- Steps must be finite and <= 5.
- No loops, no self-modifying plans.
- Prefer minimal steps that still deliver a correct answer."""

def make_plan(user_input: str) -> Plan:
    """Create a plan using OpenAI's function calling."""
    tools = [{
        "type": "function",
        "function": {
            "name": "emit_plan",
            "description": "Emit a structured execution plan",
            "parameters": Plan.model_json_schema()
        }
    }]
    msg = client.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=TEMPERATURE,
        messages=[
            {"role": "system", "content": PLANNER_SYSTEM},
            {"role": "user", "content": user_input},
        ],
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "emit_plan"}},
    )
    tool_calls = msg.choices[0].message.tool_calls or []
    if not tool_calls:
        raise ValueError("Planner did not return a plan")
    args = tool_calls[0].function.arguments
    try:
        return Plan.model_validate_json(args)
    except ValidationError as e:
        raise ValueError(f"Invalid plan schema: {e}")

def planner_node(state: AgentState) -> AgentState:
    """Planner node: creates a plan if not already present, stores it as dict."""
    if "plan_dict" in state:
        return state
    try:
        plan = make_plan(state["user_input"])
        logger.info(f"Plan created: {plan.title}")
        return {
            **state,
            "plan_dict": plan.model_dump(),
            "step_index": 0,
            "steps_results": [],
            "memory": state.get("memory", {}),
        }
    except Exception as e:
        logger.error(f"Planner error: {e}")
        return {**state, "error": {"type": "planner_error", "message": str(e)}}

**Why temperature=0?** Deterministic output. The same input should produce the same plan every time, making the system predictable and testable.

### Step 4: Build the Executor Node

The executor runs one step per visit. It validates the step's parameters, calls the tool with retries and timeout enforcement, validates the output, and appends the result to `steps_results`.

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError

class ValidationFailure(Exception):
    pass

class ToolExecutionError(Exception):
    pass

def call_tool_with_timeout(fn: Callable, inp: BaseModel, timeout_s: float) -> BaseModel:
    """Call a tool with a real timeout using ThreadPoolExecutor."""
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(fn, inp)
        try:
            return future.result(timeout=timeout_s)
        except FuturesTimeoutError:
            raise ToolExecutionError(f"Tool timed out after {timeout_s}s")

def call_tool(step_dict: Dict[str, Any], timeout_s: float = 5.0, retries: int = 2) -> Dict[str, Any]:
    """Call a tool with validation and retries."""
    name = step_dict["tool"]
    if name not in ALLOWLIST:
        raise ValidationFailure(f"Unknown tool: {name}")
    InpModel, OutModel, fn = ALLOWLIST[name]
    try:
        inp = InpModel.model_validate(step_dict["params"])
    except ValidationError as e:
        raise ValidationFailure(f"Invalid params for {name}: {e}")

    last_err = None
    for attempt in range(retries + 1):
        try:
            logger.info(f"Tool call start: {name} with {inp.model_dump()}")
            raw = call_tool_with_timeout(fn, inp, timeout_s)
            out = OutModel.model_validate(raw.model_dump())
            logger.info(f"Tool call end: {name} returned {out.model_dump()}")
            return {"tool": name, "input": inp.model_dump(), "output": out.model_dump(), "attempt": attempt}
        except Exception as e:
            last_err = e
            logger.warning(f"Tool call error: {name} attempt {attempt} failed with {e}")
            time.sleep(0.2 * (2 ** attempt))
    raise ToolExecutionError(f"Tool {name} failed after retries: {last_err}")

def executor_node(state: AgentState) -> AgentState:
    """Executor node: runs one step, validates IO, appends result."""
    plan_dict = state.get("plan_dict")
    if not plan_dict:
        return state
    steps = plan_dict["steps"]
    idx = state["step_index"]
    if idx >= len(steps):
        return state
    step = steps[idx]
    try:
        result = call_tool(step)
        new_results = [*state["steps_results"], result]
        logger.info(f"Step {idx} completed: {step['tool']}")
        return {**state, "steps_results": new_results, "step_index": idx + 1}
    except (ValidationFailure, ToolExecutionError) as e:
        logger.error(f"Executor error at step {idx}: {e}")
        return {**state, "error": {"type": "executor_error", "step": idx, "message": str(e)}}

**Why retries and timeouts?** Real-world tools can fail transiently (network glitches, rate limits). Retries with exponential backoff improve reliability. Timeouts prevent runaway calls from blocking the agent indefinitely.

### Step 5: Build the Finalizer Node

The finalizer sends the accumulated evidence to OpenAI with `response_format=json_object` to generate a concise answer. It packages the answer, evidence, plan summary, and metadata into a `FinalResult`.

In [ ]:
import json

def finalize_node(state: AgentState) -> AgentState:
    """Finalizer node: synthesizes the final answer from tool results."""
    plan_dict = state.get("plan_dict")
    if not plan_dict:
        return state
    evidence = {"steps": state["steps_results"]}
    prompt = f"""Using the tool results, answer the user succinctly.
User: {state['user_input']}
Evidence: {json.dumps(evidence, ensure_ascii=False)}"""
    try:
        msg = client.chat.completions.create(
            model=OPENAI_MODEL,
            temperature=TEMPERATURE,
            messages=[
                {"role": "system", "content": "You return JSON only with an 'answer' field."},
                {"role": "user", "content": prompt},
            ],
            response_format={"type": "json_object"},
        )
        raw = msg.choices[0].message.content
        parsed = json.loads(raw or "{}")
        final = FinalResult(
            answer=parsed.get("answer", ""),
            evidence=evidence,
            plan_summary=plan_dict,
            metadata={"model": OPENAI_MODEL, "steps": len(plan_dict["steps"])}
        )
        logger.info(f"Finalized answer: {final.answer}")
        return {**state, "final_result_dict": final.model_dump()}
    except Exception as e:
        logger.error(f"Finalizer error: {e}")
        return {**state, "error": {"type": "finalizer_error", "message": str(e)}}

**Why JSON mode?** It forces the model to return valid JSON, reducing parsing errors and ensuring structured output.

### Step 6: Add Memory Update Node

Memory runs after finalization to summarize the result and store it in thread-scoped memory. This node updates state so memory is checkpointed and available on resume.

In [ ]:
def memory_node(state: AgentState) -> AgentState:
    """Memory node: updates memory summary after finalization."""
    mem = state.get("memory", {})
    final_dict = state.get("final_result_dict")
    if final_dict:
        mem["summary"] = f"Last answer: {final_dict['answer']}"
    return {**state, "memory": mem}

**Why a separate node?** Placing memory updates inside the graph ensures they're checkpointed. If you update memory outside the graph (e.g., in the API handler), it won't persist across restarts.

### Step 7: Wire the Graph with Conditional Routing

Define the state graph, add nodes, and set up conditional edges to route between plan, execute, finalize, memory, and error handling.

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)
graph.add_node("plan", planner_node)
graph.add_node("execute", executor_node)
graph.add_node("finalize", finalize_node)
graph.add_node("memory", memory_node)

graph.set_entry_point("plan")

def route(state: AgentState):
    """Determine the next node based on the current state."""
    if "error" in state:
        return "done"  # Route to END on error
    if "final_result_dict" in state and "memory" in state:
        return "done"  # Memory already updated, finish
    if "final_result_dict" in state:
        return "memory"  # Finalized, update memory
    if "plan_dict" not in state:
        return "plan"  # No plan yet, go to planner
    plan_dict = state["plan_dict"]
    if state["step_index"] < len(plan_dict["steps"]):
        return "execute"  # More steps to run
    return "finalize"  # All steps done, finalize

graph.add_conditional_edges("plan", route, {"plan": "plan", "execute": "execute", "finalize": "finalize", "done": END})
graph.add_conditional_edges("execute", route, {"execute": "execute", "finalize": "finalize", "done": END})
graph.add_conditional_edges("finalize", route, {"memory": "memory", "done": END})
graph.add_edge("memory", END)

**Why conditional edges?** They let the graph adapt to state changes. If an error occurs, we route to END immediately. If all steps are done, we finalize. If memory is updated, we finish.

### Step 8: Compile the Graph with SQLite Checkpointer

Compile the graph with a SQLite checkpointer to enable resumability. The checkpointer persists state after every node transition.

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver.from_conn_string("checkpoints.db")
app_graph = graph.compile(checkpointer=checkpointer)

**Why SQLite?** It's portable, requires no external services, and works great for single-instance local development. For multi-instance production, swap to Postgres or Redis.

### Step 9: Wrap the Graph in a FastAPI Endpoint

Expose the agent via a `/agent` POST endpoint. The endpoint accepts a `thread_id` and `user_input`, invokes the graph with the checkpointer config, and returns the final result or error.

In [ ]:
import uuid
from fastapi import FastAPI, HTTPException

api = FastAPI(title="Deterministic Plan-Execute Agent")

class AgentRequest(BaseModel):
    thread_id: str
    user_input: str

class AgentResponse(BaseModel):
    request_id: str
    final_result: FinalResult | None = None
    error: Dict[str, Any] | None = None

@api.post("/agent", response_model=AgentResponse)
def agent_endpoint(req: AgentRequest):
    """FastAPI endpoint to handle agent requests."""
    request_id = str(uuid.uuid4())
    try:
        state_in: AgentState = {
            "thread_id": req.thread_id,
            "user_input": req.user_input,
        }
        config = {"configurable": {"thread_id": req.thread_id}}
        out: AgentState = app_graph.invoke(state_in, config=config)
        
        if "error" in out:
            return AgentResponse(request_id=request_id, error=out["error"])
        if "final_result_dict" not in out:
            raise HTTPException(status_code=500, detail="Final result missing")
        
        final_result = FinalResult.model_validate(out["final_result_dict"])
        return AgentResponse(request_id=request_id, final_result=final_result)
    except Exception as e:
        logger.exception("Unhandled error in agent endpoint")
        raise HTTPException(status_code=500, detail=str(e))

**Why FastAPI?** It's fast, has built-in Pydantic validation, and generates OpenAPI docs automatically. Perfect for exposing agents as APIs.

---

## Run and Validate

### Direct Graph Invocation (Notebook-Friendly)

Test the graph directly without starting a server. This validates the plan → execute → finalize flow with a deterministic input.

In [ ]:
# Test 1: Basic flow
state_in = {
    "thread_id": "test-thread-1",
    "user_input": "Search for LangGraph and tell me what it is."
}
config = {"configurable": {"thread_id": "test-thread-1"}}
result = app_graph.invoke(state_in, config=config)

assert "final_result_dict" in result, "Final result missing"
final = FinalResult.model_validate(result["final_result_dict"])
print(f"Answer: {final.answer}")
print(f"Steps executed: {len(final.plan_summary['steps'])}")

### Validation Error Test

Trigger a validation error by requesting an unknown tool. The graph should route to END with a structured error.

In [ ]:
# Test 2: Validation error (manually inject a bad plan)
state_bad = {
    "thread_id": "test-thread-2",
    "user_input": "Use the 'unknown_tool' to do something.",
    "plan_dict": {
        "title": "Bad Plan",
        "rationale": "Testing error handling",
        "steps": [{"tool": "unknown_tool", "params": {}, "description": "This will fail"}]
    },
    "step_index": 0,
    "steps_results": []
}
config_bad = {"configurable": {"thread_id": "test-thread-2"}}
result_bad = app_graph.invoke(state_bad, config=config_bad)

assert "error" in result_bad, "Expected error in state"
print(f"Error: {result_bad['error']}")

### Resumability Test

Simulate a mid-run failure by manually advancing the state to step 1, then resuming with the same `thread_id`. The graph should pick up where it left off.

In [ ]:
# Test 3: Resumability (simulate partial execution)
state_partial = {
    "thread_id": "test-thread-3",
    "user_input": "Search for Pydantic and FastAPI, then calculate 10 + 20.",
    "plan_dict": {
        "title": "Multi-step Plan",
        "rationale": "Test resumability",
        "steps": [
            {"tool": "search", "params": {"query": "Pydantic"}, "description": "Find Pydantic"},
            {"tool": "search", "params": {"query": "FastAPI"}, "description": "Find FastAPI"},
            {"tool": "calculator", "params": {"expression": "10 + 20"}, "description": "Calculate sum"}
        ]
    },
    "step_index": 1,  # Already completed step 0
    "steps_results": [{"tool": "search", "input": {"query": "Pydantic"}, "output": {"results": ["doc:pydantic"]}, "attempt": 0}]
}
config_partial = {"configurable": {"thread_id": "test-thread-3"}}
result_partial = app_graph.invoke(state_partial, config=config_partial)

assert result_partial["step_index"] == 3, "Should complete all steps"
print(f"Resumed and completed: {len(result_partial['steps_results'])} steps")

### API Test with Python Requests

Start the FastAPI server (in a separate terminal or notebook cell with `nest_asyncio`):

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# Run server in background (for notebook environments)
import threading
def run_server():
    uvicorn.run(api, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

import time
time.sleep(2)  # Wait for server to start

Then test the endpoint:

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/agent",
    json={"thread_id": "api-test-1", "user_input": "Search for LangGraph and summarize it."}
)
data = response.json()
print(f"Request ID: {data['request_id']}")
print(f"Answer: {data['final_result']['answer']}")

---

## Conclusion

You've built a deterministic plan-execute agent with LangGraph that:

- **Fixes the plan upfront** using OpenAI function calling and strict schemas, eliminating drift and unpredictable loops.
- **Validates every step** with Pydantic models, ensuring type safety and failing fast on bad inputs or outputs.
- **Checkpoints progress** with SQLite, enabling mid-execution resumability and resilience to transient failures.
- **Exposes a clean API** via FastAPI, making the agent easy to integrate into larger systems.
- **Maintains thread-scoped memory** so each conversation retains context without cross-contamination.

**Key design decisions:**

- **Temperature=0** for deterministic planning.
- **Tool allowlist** to prevent hallucinated or unsafe tool calls.
- **Retries and timeouts** to handle transient failures gracefully.
- **Dicts in state** for checkpoint portability (SQLite serializes to JSON).
- **Conditional routing** to handle errors and state transitions cleanly.

**Next steps:**

- **Integrate real tools:** Replace the in-memory corpus with a vector store (Pinecone, Weaviate) or live APIs (Google Search, Slack, Notion).
- **Add observability:** Instrument with LangSmith tracing or structured JSON logging to track token usage, latencies, and tool call patterns.
- **Harden for production:** Add rate limiting, circuit breakers, and health checks. Deploy with Docker and scale horizontally with a Postgres checkpointer.
- **Expand memory:** Store full conversation history and use retrieval-augmented memory for long-running threads.

You now have a production-ready foundation for building reliable, cost-predictable GenAI agents. Start with this core and iterate toward your specific use case.